In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

In [76]:
o20 = pd.read_csv('./ArgusaChallenge2023/orders_2020.csv')
o21 = pd.read_csv('./ArgusaChallenge2023/orders_2021.csv')
o22 = pd.read_csv('./ArgusaChallenge2023/orders_2022.csv')
o23 = pd.read_csv('./ArgusaChallenge2023/orders_2023.csv')
clients = pd.read_csv('./ArgusaChallenge2023/clients.csv')
ratings = pd.read_csv('./ArgusaChallenge2023/ratings.csv')
returns = pd.read_csv('./ArgusaChallenge2023/returns.csv')

products_original=pd.read_csv('./ArgusaChallenge2023/products.csv')
total_revenues = 46449091 #Information from the jury to double check

In [91]:
orders = pd.concat([o20, o21, o22, o23])

# Now we need to do a conditional merge of the orders and the products. Condition on the date
orders_merged = pd.DataFrame()
products=products_original.rename(columns={"id": "product_id"})
returned=np.zeros(len(products.index))
sold=np.zeros(len(products.index))
# Loop over product row (which means id and date range). 
# The order table subset that has that id and the date in the range, 
# will be joined with the row
for index, row in products.iterrows():
    df_to_merge = orders[(orders.order_date >= row.start_date) & (orders.order_date <= row.end_date) & (orders.product_id == row.product_id)]
    products_to_merge = products[(products.start_date == row.start_date) & (products.end_date == row.end_date) & (products.product_id == row.product_id)]
    df_to_merge = df_to_merge.merge(products_to_merge, on='product_id', validate='many_to_one')
    orders_merged = pd.concat([orders_merged, df_to_merge])

    # Counting number of sold and returned items
    sold[index]=len(df_to_merge.index)
    returned[index]=len(returns[(returns.return_date >= row.start_date) & (returns.return_date <= row.end_date) & (returns.product_id == row.product_id)])

products['returned']=returned; products['sold']=sold
orders_merged.sort_values('order_date', inplace=True, ignore_index=True)

In [92]:
# Adding also the returned items
orders_w_returns = orders_merged.merge(returns, on='order_id', how='outer')
orders = orders_w_returns

In [93]:
# Calculating some quantities on the products
products['profit'] = (products.sale_price-products.base_price)*products.sold
products['profit_margin'] = (products.sale_price-products.base_price)/products.sale_price

products['revenue'] = products.sale_price*(products.sold-products.returned)
products['net_profit'] = (products.sale_price-products['base_price'])*(products.sold-products.returned)
products['profit_ratio'] = products['net_profit']/products['revenue']

In [94]:
print(f'{products.revenue.sum()}, {total_revenues}, {products.revenue.sum()/total_revenues}')

46548509.1, 46449091, 1.0021403669664923


In [95]:
products[products.returned>products.sold]

,category_1,category_2,category_3,category_4,category_5,name,description,base_price,start_date,end_date,sale_price,product_id,returned,sold,profit,profit_margin,revenue,net_profit,profit_ratio
1520,Habitat,Gros appareils électroménagers,Laver + sécher,Machine à laver,NaN,"Sharp ES-NFW814CWA-DE, 8 kg, Gauche",Le lave-linge 8 kg ES-NFW814CWA-DE de la marqu...,465.0,2020-04-06,2020-05-13,627.97,32,13.0,11.0,1792.67,0.259519,-1255.94,-325.94,0.259519
1543,Habitat,Gros appareils électroménagers,Laver + sécher,Machine à laver,NaN,"Sharp ES-NFW814CWA-DE, 8 kg, Gauche",Le lave-linge 8 kg ES-NFW814CWA-DE de la marqu...,465.0,2022-04-01,2022-05-04,648.54,32,31.0,30.0,5506.20,0.283005,-648.54,-183.54,0.283005
4246,Animaux domestiques,Gamelle,NaN,NaN,NaN,Catit Pixi Fountain\n2.50 l,La fontaine Catit Pixi fournit à votre chat de...,40.3,2021-08-14,2021-09-01,51.71,89,27.0,25.0,285.25,0.220654,-103.42,-22.82,0.220654
4258,Animaux domestiques,Gamelle,NaN,NaN,NaN,Catit Pixi Fountain\n2.50 l,La fontaine Catit Pixi fournit à votre chat de...,40.3,2022-08-04,2022-08-25,48.32,89,26.0,24.0,192.48,0.165977,-96.64,-16.04,0.165977
4263,Animaux domestiques,Gamelle,NaN,NaN,NaN,Catit Pixi Fountain\n2.50 l,La fontaine Catit Pixi fournit à votre chat de...,40.3,2023-01-15,2023-02-02,52.35,89,28.0,21.0,253.05,0.230181,-366.45,-84.35,0.230181
4264,Animaux domestiques,Gamelle,NaN,NaN,NaN,Catit Pixi Fountain\n2.50 l,La fontaine Catit Pixi fournit à votre chat de...,40.3,2023-02-03,2023-02-17,52.44,89,26.0,17.0,206.38,0.231503,-471.96,-109.26,0.231503
4265,Animaux domestiques,Gamelle,NaN,NaN,NaN,Catit Pixi Fountain\n2.50 l,La fontaine Catit Pixi fournit à votre chat de...,40.3,2023-02-18,2023-03-04,52.19,89,27.0,20.0,237.80,0.227821,-365.33,-83.23,0.227821


In [ ]:
#### Question 2 ####
####################
total_sale = []
ids = o23_new['product_id'].unique()
for id in ids:
   sales = o23_new['price_sold'].loc[o23_new['product_id'] == id]
   total_sale.append(sales.sum())


# plot id tolat sales
fig, ax = plt.subplots()
ax.plot(ids,total_sale,"kx")
ax.set_title('Total sale per id')
ax.set_xlabel('Id')
plt.show()  


# max sales id 22
max_sale_id = ids[np.argmax(total_sale)]


# get product name
product1_name = products['name'].loc[products['id'] == max_sale_id].iloc[0]
# product name is
#Samsung QE85Q60BA, 4K, QLED, 2022, 85

In [ ]:
#### Question 3 ####
####################
categories = products['category_1'].unique()
returns_id = []
for cat in categories:
     cat_id = products.loc[products['category_1'] == cat]['id']
     returns_id.append(returns[returns['product_id'].isin(cat_id)].shape[0])
   

# Sum returns for each cat
fig, ax = plt.subplots()
ax.barh(categories, returns_id/np.sum(returns_id), height=0.5)
ax.barh(categories[1], returns_id[1]/np.sum(returns_id), color='red', height=0.6)
for i in range(np.size(returns_id)):
    if i==1: continue
    ax.text(returns_id[i]/np.sum(returns_id)+0.01, i-0.25, f'{returns_id[i]/np.sum(returns_id)*100:.1f}%')
    
ax.text(returns_id[1]/np.sum(returns_id)+0.01, 0.8, f'{returns_id[1]/np.sum(returns_id)*100:.1f}%', color='red')
#ax.set_yticklabels(categories.iloc)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.get_xaxis().set_visible(False)
ax.spines['bottom'].set_visible(False)
#ax.set_xlabel('Returned products (%)')
plt.show()

In [ ]:
#### Question 4 ####
####################
clients['n_ret_items'] = None
clients['n_buy_items'] = None
n_ret_items = np.array([])
n_buy_items = np.array([])
for year in [2020, 2021, 2022, 2023]:
    if year==2020: data=pd.DataFrame.copy(o20_new)
    elif year==2021: data=pd.DataFrame.copy(o21_new)
    elif year==2022: data=pd.DataFrame.copy(o22_new)
    elif year==2023: data=pd.DataFrame.copy(o23_new)

    client_buy = np.array([])
    client_return = np.array([])

    for ii, cl_id in enumerate(clients['client_id'].values):
        ind_ord_clients = np.where(data['client_id'].values==cl_id)[0]
        client_buy=np.append(client_buy, np.sum(data['price_sold'][ind_ord_clients].values))
        ret=0;
        for o_ret in ind_ord_clients:
            if o_ret in returns['order_id'].values:
                ret  += data['price_sold'][o_ret].values
        client_return=np.append(client_return, ret)
    if year==2020: clients['buy_2020']=client_buy; clients['ret_2020']=client_return; clients['net_2020']=client_buy-client_return
    elif year==2021: clients['buy_2021']=client_buy; clients['ret_2021']=client_return; clients['net_2021']=client_buy-client_return
    elif year==2022: clients['buy_2022']=client_buy; clients['ret_2022']=client_return; clients['net_2022']=client_buy-client_return
    elif year==2023: clients['buy_2023']=client_buy; clients['ret_2023']=client_return; clients['net_2023']=client_buy-client_return

clients['buy_total']=clients['buy_2020']+clients['buy_2021']+clients['buy_2022']+clients['buy_2023']
clients['ret_total']=clients['ret_2020']+clients['ret_2021']+clients['ret_2022']+clients['ret_2023']
clients['net_total']=clients['net_2020']+clients['net_2021']+clients['net_2022']+clients['net_2023']

In [ ]:
clients_sorted = clients.sort_values(by='net_total', ascending=False)
# Printing first three clients with ID, name and revenue
for i in [0,1,2]:
    print(clients_sorted.iloc[i]['client_id'],clients_sorted.iloc[i]['last_name'], clients_sorted.iloc[i]['fisrt_name'], clients_sorted.iloc[i]['net_total'])
    
#A4-819583b Delèze François 32148.409999999996
#V0-0674179j Jacot-Guillarmod Jonathan 24848.71
#u0-8614055c Rappaz Jean 24245.170000000002

In [ ]:
# scatter plot of their revenue
fig, ax =plt.subplots()
ax.plot(clients['net_total']*1e-3, 'k.')
ax.plot(clients_sorted['net_total'][0:2]*1e-3, 'ro', markersize=10)
ax.plot([0,np.size(clients['net_total'])], [np.mean(clients['net_total']*1e-3), np.mean(clients['net_total']*1e-3)], 'b--', linewidth=5)
ax.set_xlabel('Client ID')
ax.xaxis.set_ticklabels([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
ax.set_ylabel('Net revenue (k$)')

In [ ]:
#### Question 5 ####
####################
n_ret_items = np.array([])
n_buy_items = np.array([])
total_ret=np.zeros(np.size(clients['client_id']))
total_buy =np.zeros(np.size(clients['client_id']))
for year in [2020, 2021, 2022, 2023]:
    if year==2020: data=pd.DataFrame.copy(o20_new)
    elif year==2021: data=pd.DataFrame.copy(o21_new)
    elif year==2022: data=pd.DataFrame.copy(o22_new)
    elif year==2023: data=pd.DataFrame.copy(o23_new)

    client_buy = np.array([])
    client_return = np.array([])

    for ii, cl_id in enumerate(clients['client_id'].values):
        ind_ord_clients = np.where(data['client_id'].values==cl_id)[0]
        n_buy=np.size(ind_ord_clients)
        year_buy = np.append(client_buy, n_buy)
        
        n_ret=0;
        for o_ret in data['order_id'][ind_ord_clients]:
            if o_ret in returns['order_id'].values:
                n_ret+=1
        year_return=np.append(client_return, n_ret)
    print(np.size(total_ret), np.size(year_return))
    total_ret = total_ret+year_return
    total_buy = total_buy+year_buy

In [ ]:
print(np.sort(total_ret))